In [ ]:
import numpy as np
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

In [ ]:
loaded_data = np.load('data/GOGO/trainingdata/Play_style/organizeddata/NPZ/PS_train_dataset3_feature10_v3.npz')

# print(loaded_data.files)

x_train = loaded_data['x_train']
y_train = loaded_data['y_train']

x_train.shape ,y_train.shape

In [ ]:
def convert_to_one_hot(Y,C):
  Y = np.eye(C)[Y.reshape(-1)].T
  return Y


train_labels = convert_to_one_hot(y_train-1,3).T

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train, train_labels, test_size=0.2, random_state=1123,stratify = train_labels)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
def rotate_images_and_labels(images, labels):
    rotated_images = []
    rotated_labels = []
    
    for i, image in enumerate(images):
        rotated_90 = np.rot90(image, k=1, axes=(0, 1))
        rotated_180 = np.rot90(image, k=2, axes=(0, 1))
        rotated_270 = np.rot90(image, k=3, axes=(0, 1))
        
        rotated_images.extend([image, rotated_90, rotated_180, rotated_270])
        rotated_labels.extend([labels[i], labels[i], labels[i], labels[i]])
        
    return np.array(rotated_images), np.array(rotated_labels)

rotated_train_images, rotated_train_labels = rotate_images_and_labels(x_train, y_train)
rotated_train_images.shape, rotated_train_labels.shape

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, LayerNormalization,Softmax,Dropout,BatchNormalization ,Concatenate,Reshape,GlobalMaxPooling2D ,Add,Activation  ,multiply,Lambda,GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
import tensorflow.keras.backend as K

In [ ]:
def gelu(x):
    cdf = 0.5 * (1.0 + tf.tanh(
        (np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3)))))
    return x * cdf

In [ ]:
def inception_block(x):
    x1 = Conv2D(16, (1, 1), activation='relu', padding='same')(x)
    
    x3 = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x3 = Conv2D(192, (3, 3), activation='relu', padding='same')(x3)
    
    x9 = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x9 = Conv2D(128, (9, 9), activation='relu', padding='same')(x9)
    
    concatted  = Concatenate()([x1,x3,x9]) #,x5,x7
    
    concatted =  LayerNormalization()(concatted)
    concatted = gelu(concatted)
    return concatted

In [ ]:
def channel_attention(input_feature, ratio=8):
	
	channel_axis = 1 if K.image_data_format() == "channels_first" else -1
	channel = input_feature.shape[channel_axis]
	
	shared_layer_one = Dense(channel//ratio,
							 activation='relu',
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	shared_layer_two = Dense(channel,
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	
	avg_pool = GlobalAveragePooling2D()(input_feature)    
	avg_pool = Reshape((1,1,channel))(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	avg_pool = shared_layer_one(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel//ratio)
	avg_pool = shared_layer_two(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	
	max_pool = GlobalMaxPooling2D()(input_feature)
	max_pool = Reshape((1,1,channel))(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	max_pool = shared_layer_one(max_pool)
	assert max_pool.shape[1:] == (1,1,channel//ratio)
	max_pool = shared_layer_two(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	
	cbam_feature = Add()([avg_pool,max_pool])
	cbam_feature = Activation('sigmoid')(cbam_feature)
	
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
	
	return multiply([input_feature, cbam_feature])

In [ ]:
# 定義模型
def build_model(num_input_planes=28, k=64, num_int_conv_layers=5): 
    input_layer = Input(shape=(19, 19, 10))
    
    x = inception_block(input_layer)
    x = Dropout(0.7)(x)
    CA = channel_attention(x, 16)
    x = Dropout(0.5)(CA)
    
    x = inception_block(x)
    x = Dropout(0.7)(x)
    CA = channel_attention(x, 16)
    x = Dropout(0.5)(CA)
    
    x = inception_block(x)
    x = Dropout(0.7)(x)
    CA = channel_attention(x, 16)
    x = Dropout(0.5)(CA)

    x = Flatten()(x)

    outputs = Dense(3, activation='softmax')(x)
            
    return Model(inputs=input_layer, outputs=outputs)
    

In [ ]:
model = build_model()

model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(loss='categorical_crossentropy', optimizer= opt,metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint,CSVLogger

weight_file = '1128_f10_attention_layerNormal_v3'

val_acc='data/GOGO/trainingdata/Play_style/weight/'+ weight_file + '/weights_best_val_acc_{val_accuracy:.4f}.hdf5'
val_loss='data/GOGO/trainingdata/Play_style/weight/'+ weight_file + '/weights_best_val_loss_{val_loss:.4f}.hdf5'

checkpoint1 = ModelCheckpoint(val_acc, monitor='val_accuracy', verbose=2, save_best_only=True, save_weights_only=False, mode='max')
checkpoint2 = ModelCheckpoint(val_loss, monitor='val_loss', verbose=2, save_best_only=True, save_weights_only=False, mode='min')

csv_logger = CSVLogger('weight/Play_Style/'+ weight_file + '/training_log.csv', append=True)


callbacks_list = [checkpoint1,checkpoint2, csv_logger] 

In [ ]:
import matplotlib.pyplot as plt
def plot_training(hist,save_path):
    '''
    This function take training model and plot history of accuracy and losses with the best epoch in both of them.
    '''

    # Define needed variables
    tr_acc = hist.history['accuracy']
    tr_loss = hist.history['loss']
    val_acc = hist.history['val_accuracy']
    val_loss = hist.history['val_loss']
    index_loss = np.argmin(val_loss)
    val_lowest = val_loss[index_loss]
    index_acc = np.argmax(val_acc)
    acc_highest = val_acc[index_acc]
    Epochs = [i+1 for i in range(len(tr_acc))]
    loss_label = f'best epoch= {str(index_loss + 1)}'
    acc_label = f'best epoch= {str(index_acc + 1)}'

    # Plot training history
    plt.figure(figsize= (20, 8))
    plt.style.use('fivethirtyeight')

    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
    plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
    plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    # plt.ylim(-1,2.5)
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
    plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
    plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout
    plt.savefig('data/GOGO/trainingdata/Play_style/weight/'+ save_path)
    plt.show()

In [ ]:
from sklearn.utils import shuffle

rotated_train_images, rotated_train_labels = shuffle(rotated_train_images, rotated_train_labels, random_state=1123)

In [ ]:
history = model.fit(rotated_train_images, rotated_train_labels, epochs=100, batch_size=64,validation_split=0.2, verbose=1,callbacks=callbacks_list) #

plot_training(history,f'{weight_file}/histrory_1_1.png') 

In [ ]:
# 驗證模型
score = model.evaluate(x_test, y_test, verbose=0)

# 輸出結果
print('Test loss:', score[0])
print('Test accuracy:', score[1])